# kamAI

We will demonstrate a system that recognizes numbers from hand gestures.  
It is similar to the final project in Coursera's `Improving Deep Neural Networks: Hyperparameter tuning, Regularization and Optimization` but will be implemented using `Tensorflow 2`.  
<img src="images/hand-gestures.png">  
  
Let's start with the modules needed.

In [ ]:
import tensorflow as np
